# 25 Mistakes You Shouldn't Make

Tests and implements from [here](https://youtu.be/_gaAoJBMJ_Q)

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 01/25 - Read an CSV with an unnecessary index

In [ ]:
# The column 'id' (in this context) is an unnecessary feature. 
# We let to use just the usefull columns

usefull_columns = ['work_year', 'experience_level', 'employment_type', 'job_title', 'salary', 'salary_currency', \
                   'salary_in_usd', 'employee_residence', 'remote_ratio', 'company_location', 'company_size']

salaries_df = pd.read_csv('job_salaries.zip', compression='zip', usecols=usefull_columns)

display(salaries_df.shape, salaries_df.head())

## 02/25 - Spaces in columns

When columns has spaces isn't possible to use dot notation

In [ ]:
# Creating DataFrame
data_dict = {
    'Age Person': np.random.randint(10, 100, 200), 
    'pression value': np.random.random(200) }

data = pd.DataFrame(data = data_dict)
display(data.columns)

# To use the notation [df].[column] is necessary that the column name haven't blank spaces. 

# It's possible to alter fields names
data.columns = ['age_person', 'pression_value']
display(data.columns)

# or ...
data.rename(columns={'age_person': 'AgePerson', 'pression_value': 'PressionValue'}, inplace=True)
display(data.columns)

## 03/25 - Use Queries

In [ ]:
# Using salaries dataframe (01/25)

# Filtering work year = 2020
display(salaries_df.query("work_year == 2020").head())

# Filtering US Dollar Salaries more than the mean
salary_mean = int(salaries_df.salary_in_usd.mean())
salaries_more_than_mean = salaries_df.query(f"salary_in_usd > @salary_mean")
display(f"Salaries mean (USD) = {salary_mean}", salaries_more_than_mean.head())

## 05/25 - Use 'inplace'

In [ ]:
# data.reset_index(inplace=True)
data.rename(columns={'AgePerson': 'age_person', 'PressionValue': 'pression_value'}, inplace=True)
display(data.columns)

## 06/25 - Use vectorization

In [ ]:
# Creating a new column with criteria dependent on other 
salaries_df['is_recent'] = salaries_df['work_year'] > 2021

# then show recents ...
display(salaries_df[salaries_df['is_recent']].sample(5))

# ... and oldies
display(salaries_df[~salaries_df['is_recent']].sample(5))

## 07/25 - Use apply method

In [ ]:
# Use apply method over the vars. In this case, we calculate the exchange rate dividing the salary (anyone currency) 
# by the salary in USD, when the salary is expressed in USD, the rate will be 1.0, of course

salaries_df['exchange'] = salaries_df.apply(lambda row : row.salary/ row.salary_in_usd, axis=1)
salaries_df.sample(5)

## 08/25 - Use copy to make a new DataFrame

In [ ]:
data_recent = salaries_df.loc[salaries_df.work_year > 2021].copy()
data_recent.head()

## 09/25 - Don't create multiple intermediate dataframes 

In [ ]:
# the best choice is use enchainment 
data_out = (salaries_df.query('work_year == 2021') \
    .groupby('employment_type')[['salary_in_usd']] \
    .max().sort_values('employment_type')) \
    .reset_index().copy()

data_out.head()

## 10/25 - Always work with appropriate types

In [jobs-salaries file](./job-salaries.ipynb) we handle and cast types

In [ ]:
salaries_df.info()

## 11/25 - Use boolean instead of string values

In [ ]:
salaries_df['salary_over_100_000'] = salaries_df.apply(lambda row : 'yes' if row['salary_in_usd'] > 100_000 else 'no', axis=1)

# and then change ...
salaries_df.salary_over_100_000 = salaries_df.salary_over_100_000.map({'yes':True, 'no':False})

salaries_df.sample(10)

## 12/25 - Prefer quick plot

In [ ]:
# "Long" method
fig, ax = plt.subplots(figsize=(10,15))
plt.boxplot(salaries_df.salary_in_usd.values)
ax.set_title('Salaries')
plt.show()

In [ ]:
# Simplified new "short" method
salaries_df.boxplot(column=['salary_in_usd'])

## 13/25 - Use string method directly in array (column)

In [ ]:
salaries_df['job_title'] = list(salaries_df.job_title.str.upper())
salaries_df.sample(5)

## 14/25 - Don't repeat code

In [ ]:
# Build some dataframes (by year) and calculate the mean of salaries into each one of them
years = salaries_df.work_year.unique()
result = {}

def salary_mean(_df:pd.DataFrame, _year:int):
    result = _df.loc[_df.work_year == _year].copy()
    result['mean_no_sense'] = result.apply(lambda row : np.mean([row.salary, row.salary_in_usd]), axis=1)
    return result

dataframes = [salary_mean(salaries_df, year) for year in years]

dataframes[0].head()

## 15/25 - Rename columns

In [ ]:
# Get old columns
old_columns = salaries_df.columns.to_list()

# Create new columns
new_columns = [f"new_{col}" for col in old_columns]

# Rename columns in dataframe
salaries_df.columns = new_columns

display(old_columns, salaries_df.columns)

# Rollback
salaries_df.columns = old_columns

## 16/25 - Don't aggregate groups manually

In [ ]:
# Mean of salaries by currency
mean_salaries_by_currency = salaries_df.groupby('salary_currency')['salary_in_usd'].mean()

plt.barh(mean_salaries_by_currency.index, 
    mean_salaries_by_currency.values)

plt.show()

## 17/25 - Aggregate multiple informations

In [ ]:
aggregate_data = salaries_df.groupby('work_year')['salary_in_usd'].agg(['min','max','mean','count'])
display(aggregate_data)

## 18/25 - View changes

In [ ]:
# Add columns to follow the changes
aggregate_data['pct_change_mean'] = aggregate_data['mean'].pct_change()
aggregate_data['diff_mean'] = aggregate_data['mean'].diff()

display(aggregate_data)

## 19/25 - Don't save in _CSV_ when is not necessary; use more efficient formats (like _parquet_)

In [ ]:
salaries_df.to_parquet('salaries.parquet')

In [ ]:
%%time
# The parquet file have more data
result = pd.read_parquet('salaries.parquet')

## 20/25 - Use styles in pandas dataframe

In [ ]:
salaries_df.sort_values(['work_year', 'salary_in_usd']) \
    .head(15)[['salary_in_usd']] \
    .reset_index(drop=True) \
    .style.background_gradient(cmap='Reds')

## 21/25 - Do not use _x and _y when merging 2 datasets ...
### ... prefer to use words from the context (domain)

In [ ]:
# The var 'dataframes' was declared in the 14 tip
# Warning! This script don't make sense!!

salaries_merged = dataframes[0].merge(dataframes[1], on=['job_title'], suffixes=('_2020', '_2021'))
salaries_merged.sample(10)

## 22/25 - Count result from merge
### ... and use 'assert' to test!

In [ ]:
assert len(salaries_merged) == 1606, "The len of 'salaries_merged' must be equal 1606"

## 23/25 - Staking change command in one line of code

In [ ]:
salaries_df2 = (salaries_df.sort_values('job_title', ascending=False)
    .groupby('work_year'))

display(salaries_df2.head())

## 24/25 - Prefer 'category' as a data type to spend less memory

In [ ]:
d2 = salaries_df[['employment_type']].copy()
d2.info()

In [ ]:
d2['employment_type'] = d2['employment_type'].astype('category')
d2.info()

## 25/25 - Remove or avoid duplicated columns

In [ ]:
df_duplicate = pd.concat([salaries_df, dataframes[1]], axis=1)
df_duplicate = df_duplicate.loc[:,~df_duplicate.columns.duplicated()].copy()
df_duplicate['job_title'].head()